In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# BoundsEnforceLS

The BoundsEnforceLS only backtracks until variables violate their upper and lower bounds.

Here is a simple example where BoundsEnforceLS is used to backtrack during the Newton solver's iteration on
a system that contains an implicit component with 3 states that are confined to a small range of values.

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src20", get_code("openmdao.test_suite.components.implicit_newton_linesearch.ImplCompTwoStatesArrays"), display=False)

:::{Admonition} `ImplCompTwoStatesArrays` class definition 
:class: dropdown

{glue:}`code_src20`
:::

In [ ]:
import numpy as np

import openmdao.api as om
from openmdao.test_suite.components.implicit_newton_linesearch import ImplCompTwoStatesArrays

top = om.Problem()
top.model.add_subsystem('comp', ImplCompTwoStatesArrays(), promotes_inputs=['x'])

top.model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
top.model.nonlinear_solver.options['maxiter'] = 10
top.model.linear_solver = om.ScipyKrylov()

top.model.nonlinear_solver.linesearch = om.BoundsEnforceLS()

top.setup()
top.set_val('x', np.array([2., 2, 2]).reshape(3, 1))

# Test lower bounds: should go to the lower bound and stall
top.set_val('comp.y', 0.)
top.set_val('comp.z', 1.6)
top.run_model()

for ind in range(3):
    print(top.get_val('comp.z', indices=ind))

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal

for ind in range(3):
    assert_near_equal(top.get_val('comp.z', indices=ind), [1.5], 1e-8)

## BoundsEnforceLS Options

In [ ]:
om.show_options_table("openmdao.solvers.linesearch.backtracking.BoundsEnforceLS")

## BoundsEnforceLS Constructor

The call signature for the `BoundsEnforceLS` constructor is:

```{eval-rst}
    .. automethod:: openmdao.solvers.linesearch.backtracking.BoundsEnforceLS.__init__
        :noindex:
```

## BoundsEnforceLS Option Examples

**bound_enforcement**

BoundsEnforceLS includes the `bound_enforcement` option in its options dictionary. This option has a dual role:

1. Behavior of the non-bounded variables when the bounded ones are capped.
2. Direction of the further backtracking.

There are three difference bounds enforcement schemes available in this option.

With "scalar" bounds enforcement, only the variables that violate their bounds are pulled back to feasible values; the
remaining values are kept at the Newton-stepped point. This changes the direction of the backtracking vector so that
it still moves in the direction of the initial point. This is the default bounds enforcement for `BoundsEnforceLS`.

![BT2](images/BT2.jpg)

With "vector" bounds enforcement, the solution in the output vector is pulled back in unison to a point where none of the
variables violate any upper or lower bounds. Further backtracking continues along the Newton gradient direction vector back towards the
initial point.

![BT1](images/BT1.jpg)

With "wall" bounds enforcement, only the variables that violate their bounds are pulled back to feasible values; the remaining values are kept at the Newton-stepped point. Further backtracking only occurs in the direction of the non-violating variables, so that it will move along the wall.

```{Note}
When using BoundsEnforceLS linesearch, the `scalar` and `wall` methods are exactly the same because no further
backtracking is performed.
```

![BT3](images/BT3.jpg)

Here are a few examples of this option:

- bound_enforcement: vector

  The `bound_enforcement` option in the options dictionary is used to specify how the output bounds
  are enforced. When this is set to "vector", the output vector is rolled back along the computed gradient until
  it reaches a point where the earliest bound violation occurred. The backtracking continues along the original
  computed gradient.

In [ ]:
from openmdao.test_suite.components.implicit_newton_linesearch import ImplCompTwoStatesArrays

top = om.Problem()
top.model.add_subsystem('comp', ImplCompTwoStatesArrays(), promotes_inputs=['x'])

top.model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
top.model.nonlinear_solver.options['maxiter'] = 10
top.model.linear_solver = om.ScipyKrylov()

top.model.nonlinear_solver.linesearch = om.BoundsEnforceLS(bound_enforcement='vector')

top.setup()
top.set_val('x', np.array([2., 2, 2]).reshape(3, 1))

# Test lower bounds: should go to the lower bound and stall
top.set_val('comp.y', 0.)
top.set_val('comp.z', 1.6)
top.run_model()

for ind in range(3):
    print(top.get_val('comp.z', indices=ind))

In [ ]:
for ind in range(3):
    assert_near_equal(top.get_val('comp.z', indices=ind), [1.5], 1e-8)

- bound_enforcement: scalar

  The `bound_enforcement` option in the options dictionary is used to specify how the output bounds
  are enforced. When this is set to "scaler", then the only indices in the output vector that are rolled back
  are the ones that violate their upper or lower bounds. The backtracking continues along the modified gradient.

In [ ]:
from openmdao.test_suite.components.implicit_newton_linesearch import ImplCompTwoStatesArrays

top = om.Problem()
top.model.add_subsystem('comp', ImplCompTwoStatesArrays(), promotes_inputs=['x'])

top.model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
top.model.nonlinear_solver.options['maxiter'] = 10
top.model.linear_solver = om.ScipyKrylov()

top.model.nonlinear_solver.linesearch = om.BoundsEnforceLS(bound_enforcement='scalar')

top.setup()
top.set_val('x', np.array([2., 2, 2]).reshape(3, 1))
top.run_model()

In [ ]:
# Test lower bounds: should stop just short of the lower bound
top.set_val('comp.y', 0.)
top.set_val('comp.z', 1.6)
top.run_model()

- bound_enforcement: wall

  The `bound_enforcement` option in the options dictionary is used to specify how the output bounds
  are enforced. When this is set to "wall", then the only indices in the output vector that are rolled back
  are the ones that violate their upper or lower bounds. The backtracking continues along a modified gradient
  direction that follows the boundary of the violated output bounds.

In [ ]:
from openmdao.test_suite.components.implicit_newton_linesearch import ImplCompTwoStatesArrays

top = om.Problem()
top.model.add_subsystem('comp', ImplCompTwoStatesArrays(), promotes_inputs=['x'])

top.model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
top.model.nonlinear_solver.options['maxiter'] = 10
top.model.linear_solver = om.ScipyKrylov()

top.model.nonlinear_solver.linesearch = om.BoundsEnforceLS(bound_enforcement='wall')

top.setup()
top.set_val('x', np.array([0.5, 0.5, 0.5]).reshape(3, 1))

# Test upper bounds: should go to the upper bound and stall
top.set_val('comp.y', 0.)
top.set_val('comp.z', 2.4)
top.run_model()

print(top.get_val('comp.z', indices=0))
print(top.get_val('comp.z', indices=1))
print(top.get_val('comp.z', indices=2))

In [ ]:
assert_near_equal(top.get_val('comp.z', indices=0), [2.6], 1e-8)
assert_near_equal(top.get_val('comp.z', indices=1), [2.5], 1e-8)
assert_near_equal(top.get_val('comp.z', indices=2), [2.65], 1e-8)